## BalancedEncoderDecoder

In [1]:
!pwd

/home/st/sbhuvaji/Research/Balanced


In [2]:
import socket
hostname = socket.gethostname()
ip_address = socket.gethostbyname(hostname)

# Print the SSH address
ssh_address = f"{hostname}@{ip_address}"
print(f"SSH Address: {ssh_address}")

SSH Address: csp239.cslab.seattleu.edu@10.124.66.239


In [ ]:
#Disable Warning
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

In [4]:
import tensorflow as tf
# Get the list of available GPUs
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print("Available GPUs:")
    for gpu in gpus:
        print(gpu)

Available GPUs:
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


In [5]:
%%capture
!pip install -U gretel-client

In [6]:
FILE1 = '../SB-001/SB-001_normalized_AM.csv'
FILE2 = '../SB-001/SB-001_normalized_PM.csv'
NO_OF_COLUMNS = 24

In [7]:
#!pip install tf2onnx

import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
plt.ioff()
import cv2
import IPython
from tqdm import tqdm
import time
import os
import pickle
import pandas as pd
from gretel_client.config import RunnerMode
from gretel_client.evaluation.quality_report import QualityReport
from gretel_client import configure_session
from gretel_client.projects import create_or_get_unique_project
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', message=r'.*Consider either turning off auto-sharding.*')

In [8]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"  # GPU Id

In [9]:
# get your gretel API key ready befor running this
import pandas as pd
from gretel_client.config import RunnerMode
from gretel_client.evaluation.quality_report import QualityReport
from gretel_client import configure_session
from gretel_client.projects import create_or_get_unique_project

# Specify your Gretel API Key

pd.set_option("max_colwidth", None)
configure_session(api_key="prompt", cache="yes", validate=True)
run_summary = []

Using endpoint https://api.gretel.cloud
Logged in as sbhuvaj@gmail.com ✅


In [10]:
def generate_autoencoder_reports(encoder_dense_layers, bottle_neck, decoder_dense_layers):

    print(encoder_dense_layers, bottle_neck, decoder_dense_layers)
    # Load Data
    df_am = pd.read_csv(FILE1)
    df_pm = pd.read_csv(FILE2)
    combined_df = pd.concat([df_am, df_pm])

    columns_needed = ['y_am_pef', 'tempin', 'humidin', 'pm25in', 'co2in', 'tempdiffin', 'humidiffin', 'pm25diffin',
                      'pm10', 'pm25', 'o3', 'no2', 'co', 'so2', 'temp', 'windsd', 'humid', 'varp', 'dewpt', 'airp',
                      'seap', 'solrhr', 'solramnt', 'grdt', 'class']
    combined_df = combined_df.filter(columns_needed)
    df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

    # Drop NaN
    df = df.dropna()
    
    # Minotity Split
    class_counts = df['class'].value_counts()
    minority_class = df[df['class'] == 0]
    majority_class = df[df['class'] == 1]

    # Removing class column from minority before augmentation
    X = minority_class.drop('class', axis=1)
    y = minority_class['class']

    # Saving minority as X_train
    X_train = X

    # Define the Input shape
    INPUT_SHAPE = X_train.shape[1]
    FILE_NAME = f"L{INPUT_SHAPE}_E{'_'.join(map(str, encoder_dense_layers))}_B{bottle_neck}_D{'_'.join(map(str, decoder_dense_layers))}"
    print("\n" + FILE_NAME + "\n")
    
    def build_autoencoder(input_shape, **kwargs):

        # Autoencoder model construction cod

      """
      Build an autoencoder model with the given configuration.

      Args:
          input_shape (int): The shape of the input data.
          **kwargs: Additional keyword arguments for configuring the autoencoder.

      Keyword Args:
          encoder_dense_layers (list): List of integers specifying the number of units for each dense layer in the encoder.
                                      Default: []
          bottle_neck (int): The number of units in the bottleneck layer. Default: input_shape // 2
          decoder_dense_layers (list): List of integers specifying the number of units for each dense layer in the decoder.
                                      Default: []
          decoder_activation (str): The activation function for the decoder output layer. Default: 'sigmoid'

      Returns:
          tuple: A tuple containing the autoencoder, encoder, and decoder models.

      Example:
          INPUT_SHAPE = 27
          encoder_dense_layers = [20, 18]  # Specify the number of units for each dense layer in the encoder
          bottle_neck = 16
          decoder_dense_layers = [18, 20]  # Specify the number of units for each dense layer in the decoder

          autoencoder, encoder, decoder = build_autoencoder(INPUT_SHAPE, encoder_dense_layers=encoder_dense_layers,
                                                            bottle_neck=bottle_neck, decoder_dense_layers=decoder_dense_layers)
          encoder.summary()
          decoder.summary()
          autoencoder.summary()
      """

      # Default parameter values
      encoder_dense_layers = kwargs.get('encoder_dense_layers', [])
      bottle_neck = kwargs.get('bottle_neck', input_shape // 2)
      decoder_dense_layers = kwargs.get('decoder_dense_layers', [])
      decoder_activation = kwargs.get('decoder_activation', 'sigmoid')

      # Autoencoder Model
      encoder_input = keras.Input(shape=(input_shape,), name="encoder")
      x = keras.layers.Flatten()(encoder_input)

      # Encoder Dense Layers
      for units in encoder_dense_layers:
          x = keras.layers.Dense(units, activation="relu")(x)

      encoder_output = keras.layers.Dense(bottle_neck, activation="relu")(x)
      encoder = keras.Model(encoder_input, encoder_output, name="encoder")

      # Decoder Model
      decoder_input = keras.Input(shape=(bottle_neck,), name="decoder")
      x = decoder_input

      # Decoder Dense Layers
      for units in decoder_dense_layers:
          x = keras.layers.Dense(units, activation="relu")(x)

      decoder_output = keras.layers.Dense(input_shape, activation=decoder_activation)(x)
      decoder = keras.Model(decoder_input, decoder_output, name="decoder")

      # Autoencoder Model
      autoencoder_input = keras.Input(shape=(input_shape,), name="input")
      encoded = encoder(autoencoder_input)
      decoded = decoder(encoded)
      autoencoder = keras.Model(autoencoder_input, decoded, name="autoencoder")

      return autoencoder, encoder, decoder

    physical_devices = tf.config.list_physical_devices("GPU")
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    tf.config.experimental.set_memory_growth(physical_devices[1], True)
    strategy = tf.distribute.MirroredStrategy()
    
    with strategy.scope():
        autoencoder, encoder, decoder = build_autoencoder(INPUT_SHAPE, encoder_dense_layers=encoder_dense_layers,
                                                          bottle_neck=bottle_neck,
                                                          decoder_dense_layers=decoder_dense_layers)
        opt = keras.optimizers.Adam(learning_rate=0.001)
        autoencoder.compile(opt, loss="mse")

        print("Training model:", FILE_NAME)
        history = autoencoder.fit(X_train, X_train, epochs=200, batch_size=16, validation_split=0.25, verbose=0)
        print("Training Complete:")
    
    # Extract the loss values
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Print the last epoch's loss values
    last_epoch_loss = loss[-1]
    last_epoch_val_loss = val_loss[-1]
    print("Last epoch loss:", last_epoch_loss)
    print("Last epoch validation loss:", last_epoch_val_loss)

    # Saving history
    with open(FILE_NAME + 'history.pickle', 'wb') as file:
        pickle.dump(history.history, file)

    # Visualize losses *(MSE)
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(train_loss) + 1)
    plt.plot(epochs, train_loss, 'b', label='Training Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(FILE_NAME + 'Loss.png')
    #plt.show()

    # Generate synthetic data
    num_samples = len(X_train)
    input_data = np.random.normal(size=(num_samples, INPUT_SHAPE))
    generated_data = autoencoder.predict(input_data)
    reshaped_data = generated_data.reshape(num_samples, -1)
    df_generated = pd.DataFrame(reshaped_data, columns=X_train.columns)

    # Plot correlation matrices
    corr_matrix1 = X_train.corr()
    corr_matrix2 = df_generated.corr()

    fig, axes = plt.subplots(1, 2, figsize=(18, 10))
    sns.heatmap(corr_matrix1, annot=True, cmap='coolwarm', ax=axes[0])
    axes[0].set_title('Heatmap 1')

    sns.heatmap(corr_matrix2, annot=True, cmap='coolwarm', ax=axes[1])
    axes[1].set_title('Heatmap 2')

    plt.tight_layout()
    plt.savefig(FILE_NAME + 'HeatMaps')
    #plt.show()

    # Calculate mean and standard deviation of original and synthetic datasets
    common_columns = set(X_train.columns) & set(df_generated.columns)
    results = {}

    for column in common_columns:
        mean_df1 = X_train[column].mean()
        std_df1 = X_train[column].std()
        mean_df2 = df_generated[column].mean()
        std_df2 = df_generated[column].std()

        results[column] = {'Mean_df1': mean_df1, 'Std_df1': std_df1,
                           'Mean_df2': mean_df2, 'Std_df2': std_df2}

    comparison_df = pd.DataFrame(results)
    comparison_df.to_csv(FILE_NAME + 'mean_std.csv', index=True)

    # Plot scatter plots
    new_index = pd.RangeIndex(start=0, stop=57, step=1)
    X_train = X_train.set_index(new_index)
    common_columns = set(X_train.columns) & set(df_generated.columns)

    for column in common_columns:
        plt.scatter(X_train.index, X_train[column], color='red', label='X_train')
        plt.scatter(df_generated.index, df_generated[column], color='blue', label='df_generated')

        plt.title(f"Scatter Plot: {column}")
        plt.xlabel("Index")
        plt.ylabel(column)

        plt.legend()
        plt.savefig(FILE_NAME + "_" + column)
        #plt.show()


    # Add back the class label
    X_train['class'] = 0.0
    df_generated['class'] = 0.0
    X_train.to_csv(FILE_NAME + '_Original_minority_data.csv', index=False)
    df_generated.to_csv(FILE_NAME + '_Synthetic_minority_data.csv', index=False)

#     # Generate quality report and display it
#     real_data = FILE_NAME + 'Original_minority_data.csv'
#     synth_data = FILE_NAME + 'Synthetic_minority_data.csv'
#
#     report = QualityReport(data_source=synth_data, ref_data=real_data)
#     report.run()

#     print(report.peek())
#    run_summary.append([FILE_NAME, report.peek()['raw_score'], report.peek()['grade']])
    run_summary.append([FILE_NAME, last_epoch_loss, last_epoch_val_loss])
    print("Reports Saved")
    #IPython.display.HTML(report.as_html, metadata=dict(isolated=True))
    #return report
    return None

In [11]:
# Example usage
def runner(encoder_dense_layers, bottle_neck, decoder_dense_layers):
  #encoder_dense_layers = [10, 8]
  #bottle_neck = 5
  #decoder_dense_layers = [8, 10]
  #FILE_NAME = f"24_{'_'.join(map(str, encoder_dense_layers))}_{bottle_neck}_"
  FILE_NAME = f"L{NO_OF_COLUMNS}_E{'_'.join(map(str, encoder_dense_layers))}_B{bottle_neck}_D{'_'.join(map(str, decoder_dense_layers))}"   
  report = generate_autoencoder_reports(encoder_dense_layers, bottle_neck, decoder_dense_layers)
  #print(type(report))
  # Save the report object to a file
#   with open(FILE_NAME + 'quality_report.pickle', 'wb') as file:
#       pickle.dump(report, file)

  #IPython.display.HTML(report.as_html, metadata=dict(isolated=True))

In [12]:
# here goes nothin

In [13]:
encoder_dense_layers_trial = [[10, 8], [12, 10], [14, 12], [16, 14], [18, 16], [20, 18] ,[22,20]]
decoder_dense_layers_trial = [[8, 10], [10, 12], [12, 14], [14, 16], [16, 18], [18, 20], [20,22]]
bottle_neck_trial = [8, 10, 12, 14, 16, 18]

In [14]:
total_iterations = len(bottle_neck_trial) * len(encoder_dense_layers_trial) * len(decoder_dense_layers_trial)
print("Total Model in Pipeline:",total_iterations)

#print total models
with tqdm(total=total_iterations, desc="Overall Progress") as pbar:
    for bn in bottle_neck_trial:
        for enc_layers in encoder_dense_layers_trial:
            for dec_layers in decoder_dense_layers_trial:
                pbar.set_postfix({"Encoder Dense Layers": enc_layers, "Decoder Dense Layers": dec_layers, "Bottle Neck": bn})
                print(enc_layers, bn, dec_layers)
                pbar.update(1)

               


# encoder_dense_layers_trial = [[10, 8], [12, 10], [14, 12], [16, 14], [18, 16], [20, 18] ,[22,20]]
# decoder_dense_layers_trial = [[8, 10], [10, 12], [12, 14], [14, 16], [16, 18], [18, 20], [20,22]]
# bottle_neck_trial = [8, 10, 12, 14, 16, 18]

# for bn in bottle_neck_trial:
#     for enc_layers, dec_layers in zip(encoder_dense_layers_trial, decoder_dense_layers_trial):
#         print(f"Encoder Dense Layers: {enc_layers}, Decoder Dense Layers: {dec_layers}, Bottle Neck: {bn}")
#         runner(enc_layers, bn, dec_layers)
#         print()

Total Model in Pipeline: 294


Overall Progress:  13%|███████▋                                                   | 38/294 [00:00<00:00, 2087.32it/s, Encoder Dense Layers=[20, 18], Decoder Dense Layers=[14, 16], Bottle Neck=8]

[10, 8] 8 [8, 10]
[10, 8] 8 [10, 12]
[10, 8] 8 [12, 14]
[10, 8] 8 [14, 16]
[10, 8] 8 [16, 18]
[10, 8] 8 [18, 20]
[10, 8] 8 [20, 22]
[12, 10] 8 [8, 10]
[12, 10] 8 [10, 12]
[12, 10] 8 [12, 14]
[12, 10] 8 [14, 16]
[12, 10] 8 [16, 18]
[12, 10] 8 [18, 20]
[12, 10] 8 [20, 22]
[14, 12] 8 [8, 10]
[14, 12] 8 [10, 12]
[14, 12] 8 [12, 14]
[14, 12] 8 [14, 16]
[14, 12] 8 [16, 18]
[14, 12] 8 [18, 20]
[14, 12] 8 [20, 22]
[16, 14] 8 [8, 10]
[16, 14] 8 [10, 12]
[16, 14] 8 [12, 14]
[16, 14] 8 [14, 16]
[16, 14] 8 [16, 18]
[16, 14] 8 [18, 20]
[16, 14] 8 [20, 22]
[18, 16] 8 [8, 10]
[18, 16] 8 [10, 12]
[18, 16] 8 [12, 14]
[18, 16] 8 [14, 16]
[18, 16] 8 [16, 18]
[18, 16] 8 [18, 20]
[18, 16] 8 [20, 22]
[20, 18] 8 [8, 10]
[20, 18] 8 [10, 12]
[20, 18] 8 [12, 14]


Overall Progress: 100%|█████████████████████████████████████████████████████████| 294/294 [00:00<00:00, 1994.93it/s, Encoder Dense Layers=[22, 20], Decoder Dense Layers=[20, 22], Bottle Neck=18]

[20, 18] 8 [14, 16]
[20, 18] 8 [16, 18]
[20, 18] 8 [18, 20]
[20, 18] 8 [20, 22]
[22, 20] 8 [8, 10]
[22, 20] 8 [10, 12]
[22, 20] 8 [12, 14]
[22, 20] 8 [14, 16]
[22, 20] 8 [16, 18]
[22, 20] 8 [18, 20]
[22, 20] 8 [20, 22]
[10, 8] 10 [8, 10]
[10, 8] 10 [10, 12]
[10, 8] 10 [12, 14]
[10, 8] 10 [14, 16]
[10, 8] 10 [16, 18]
[10, 8] 10 [18, 20]
[10, 8] 10 [20, 22]
[12, 10] 10 [8, 10]
[12, 10] 10 [10, 12]
[12, 10] 10 [12, 14]
[12, 10] 10 [14, 16]
[12, 10] 10 [16, 18]
[12, 10] 10 [18, 20]
[12, 10] 10 [20, 22]
[14, 12] 10 [8, 10]
[14, 12] 10 [10, 12]
[14, 12] 10 [12, 14]
[14, 12] 10 [14, 16]
[14, 12] 10 [16, 18]
[14, 12] 10 [18, 20]
[14, 12] 10 [20, 22]
[16, 14] 10 [8, 10]
[16, 14] 10 [10, 12]
[16, 14] 10 [12, 14]
[16, 14] 10 [14, 16]
[16, 14] 10 [16, 18]
[16, 14] 10 [18, 20]
[16, 14] 10 [20, 22]
[18, 16] 10 [8, 10]
[18, 16] 10 [10, 12]
[18, 16] 10 [12, 14]
[18, 16] 10 [14, 16]
[18, 16] 10 [16, 18]
[18, 16] 10 [18, 20]
[18, 16] 10 [20, 22]
[20, 18] 10 [8, 10]
[20, 18] 10 [10, 12]
[20, 18] 10 [12, 

In [ ]:
start_time = time.time()

with tqdm(total=total_iterations, desc="Overall Progress") as pbar:
    for bn in bottle_neck_trial:
        for enc_layers in encoder_dense_layers_trial:
            for dec_layers in decoder_dense_layers_trial:
                pbar.set_postfix({"Encoder Dense Layers": enc_layers, "Decoder Dense Layers": dec_layers, "Bottle Neck": bn})
                runner(enc_layers, bn, dec_layers)
                pbar.update(1) 
                
end_time = time.time()
total_time = end_time - start_time                
total_time_minutes = total_time / 60
print(f"Total time: {total_time_minutes} minutes") 

Overall Progress:   0%|                                                                        | 0/294 [00:00<?, ?it/s, Encoder Dense Layers=[10, 8], Decoder Dense Layers=[8, 10], Bottle Neck=8]

[10, 8] 8 [8, 10]

L24_E10_8_B8_D8_10

Training model: L24_E10_8_B8_D8_10
Training Complete:
Last epoch loss: 0.01919788122177124
Last epoch validation loss: 0.028741061687469482


Overall Progress:   0%|▏                                                            | 1/294 [01:21<6:40:01, 81.92s/it, Encoder Dense Layers=[10, 8], Decoder Dense Layers=[10, 12], Bottle Neck=8]

Reports Saved
[10, 8] 8 [10, 12]

L24_E10_8_B8_D10_12

Training model: L24_E10_8_B8_D10_12
Training Complete:
Last epoch loss: 0.015513232909142971
Last epoch validation loss: 0.021702827885746956


Overall Progress:   1%|▍                                                            | 2/294 [02:42<6:34:57, 81.16s/it, Encoder Dense Layers=[10, 8], Decoder Dense Layers=[12, 14], Bottle Neck=8]

...

Reports Saved
[14, 12] 18 [16, 18]

L24_E14_12_B18_D16_18

Training model: L24_E14_12_B18_D16_18
Training Complete:
Last epoch loss: 0.008093859069049358
Last epoch validation loss: 0.014677663333714008


Overall Progress:  90%|███████████████████████████████████████████████████▏     | 264/294 [5:53:21<42:19, 84.64s/it, Encoder Dense Layers=[14, 12], Decoder Dense Layers=[18, 20], Bottle Neck=18]

Reports Saved
[14, 12] 18 [18, 20]

L24_E14_12_B18_D18_20

Training model: L24_E14_12_B18_D18_20
Training Complete:
Last epoch loss: 0.011336835101246834
Last epoch validation loss: 0.018763471394777298


Overall Progress:  90%|███████████████████████████████████████████████████▍     | 265/294 [5:54:45<40:51, 84.53s/it, Encoder Dense Layers=[14, 12], Decoder Dense Layers=[20, 22], Bottle Neck=18]

Reports Saved
[14, 12] 18 [20, 22]

L24_E14_12_B18_D20_22

Training model: L24_E14_12_B18_D20_22
Training Complete:
Last epoch loss: 0.01040484569966793
Last epoch validation loss: 0.01829705946147442


In [ ]:
print(run_summary)

In [ ]:
#save run_summary as a file
filename = 'run_summary.csv'

with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(run_summary)

print(f"Run summary has saved: {filename}")

In [ ]:
run_summary.sort(key=lambda x: x[1], reverse=True)  # Sort by the middle value in descending order

top_10 = run_summary[:10]  # Get the top 10 elements

for item in top_10:
    print(item)